# PyMC Marketing Colab Notebook  
*Hands‑on MMM & CLV in Python*

In [ ]:
!pip install pymc-marketing[extra] --quiet

## Verify installation

In [ ]:
import pymc_marketing as pmm
print('PyMC Marketing:', pmm.__version__)

## Generate synthetic weekly dataset

In [ ]:
import numpy as np, pandas as pd, arviz as az
from pymc_marketing.mmm.utils import add_fourier_terms

rng = np.random.default_rng(42)
dates = pd.date_range('2018-04-01', '2021-09-01', freq='W-MON')
df = pd.DataFrame({'date_week': dates})
df['x1'] = rng.uniform(0, 1, len(df))
df['x2'] = rng.uniform(0, 1, len(df))
df['event_1'] = (df['date_week'] == '2019-05-13').astype(int)
df['event_2'] = (df['date_week'] == '2020-09-14').astype(int)
df['t'] = np.arange(len(df))
df = add_fourier_terms(df, date_col='date_week', period=52, order=2)
df.head()

## Specify and fit MMM

In [ ]:
from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from pymc_marketing.prior import Prior

model_cfg = {
    "intercept": Prior("Normal", mu=0.5, sigma=0.2),
    "saturation_beta": Prior("HalfNormal", sigma=[1.0, 1.0]),
    "gamma_control": Prior("Normal", mu=0, sigma=0.05),
    "gamma_fourier": Prior("Laplace", mu=0, b=0.2),
    "likelihood": Prior("Normal", sigma=Prior("HalfNormal", sigma=6)),
}

mmm = MMM(
    model_config=model_cfg,
    date_column="date_week",
    adstock=GeometricAdstock(l_max=8),
    saturation=LogisticSaturation(),
    channel_columns=["x1", "x2"],
    control_columns=["event_1", "event_2", "t"],
    yearly_seasonality=2,
)

X = df[['x1', 'x2', 'event_1', 'event_2', 't']]
y = rng.normal(loc=10 + 3*df['x1'] + 2*df['x2'], scale=1.0)

mmm.fit(X, y,
        chains=4,
        target_accept=0.9,
        nuts_sampler='numpyro',
        random_seed=rng)

## Diagnostics & posterior predictive

In [ ]:
import matplotlib.pyplot as plt
mmm.plot_posterior_predictive(original_scale=True)
plt.show()
mmm.plot_components_contributions(original_scale=True)
plt.show()

## Forecasting scenario

In [ ]:
future = df.tail(8).copy()
future['x1'] *= 1.2
future['x2'] *= 0.8
forecast = mmm.predict(future, include_last_observations=True)
forecast.head()